In [132]:
import random
import numpy as np
import pulp

def generate(k, n):
    P = np.random.randint(-100, 100, size=(k, n-k))
    I = np.eye(k)
    G = np.hstack((I, P))
    Psi = np.random.choice([-1,1], size=n-k) # m dependant for m in M: Psi_=Psi[:m]
    Y = list(range(n))
    a = Y.pop(random.randrange(len(Y)))
    b = Y.pop(random.randrange(len(Y)))
    X = np.array([]) # X starts empty and gets filled to the initial m-1=1 at the first iteration of LPsolver and increases every pass
    return G, P, Psi, a, b, X, Y

In [228]:
DEBUG = 0
def solve(n, k):
    M=list(range(n-k+1))[2:]
    print(f'solving n,k = {n,k}')
    print(f'over m = {M}')
    attempts = 0
    bounded = 0
    while True:
        if attempts%5000==0: print(f'{attempts} attempts reached\n{bounded} bounded solutions found')
        G, P, Psi, a, b, X, Y = generate(k, n)
        attempts += 1
        results = np.array([])
        for m in M:
            t = Y.pop(random.randrange(len(Y)))
            X = np.append(X, t)
            X = sorted(X)
            r = LPsolver(n, k, G, Psi[:m], a, b, X, Y)
            if r==-1:break
            bounded += 1
            results = np.append(results, r)
        if r==-1: continue
        print('G:\n',G)
        print('Ψ',Psi)
        print('a,b',a,b)
        print('X',X)
        print('Y',Y)
        return P, results
    print(f'{bounded} bounded soulutions in {attempts} attempts')
    return -1
def LPsolver(n, k, G, Psi, a, b, X, Y):
    # Define the LP maximization problem
    prob = pulp.LpProblem("Maximize", pulp.LpMaximize)

    # Create decision variables: u_i (assume non-negative; adjust bounds if necessary)
    U = [pulp.LpVariable(f"u_{i}", lowBound=0) for i in range(k)]
    # tau = [a] + X + [b] + Y
    # Objective function: maximize sum_i s0 * g[i] * u_i
    prob += pulp.lpSum(Psi[0] * G[i,a] * U[i] for i in range(k)), "Objective"
    for idx,j in enumerate(X):
        j = int(j)
        prob += pulp.lpSum((Psi[idx+1] * G[i,j] - Psi[0] * G[i,a]) * U[i] for i in range(k)) <= 0, f"Constraint1_{j}"
        prob += pulp.lpSum(-Psi[idx+1] * G[i,j] * U[i] for i in range(k)) <= -1, f"Constraint2_{j}"
    prob += pulp.lpSum(G[i,b] * U[i] for i in range(k)) == 1, f"Constraint3"
    for j in Y:
        j = int(j)
        prob += pulp.lpSum(G[i,j] * U[i] for i in range(k)) <= 1, f"Constraint4_{j}"
        prob += pulp.lpSum(-G[i,j] * U[i] for i in range(k)) <= 1, f"Constraint5_{j}"

    # Solve the LP
    result_status = prob.solve()
    if DEBUG:
        print("Status:", pulp.LpStatus[result_status])
        # print("Optimal Objective Value:", pulp.value(prob.objective))
    
    if pulp.LpStatus[result_status] == "Optimal":
    # Extract and print solution for U.
        solution_U = [pulp.value(u) for u in U]
        if DEBUG:
        # print("Solution U:", solution_U)
            print("Optimal Objective Value:", pulp.value(prob.objective))
            print('G:\n',G)
            print('Ψ',Psi)
            print('a,b',a,b)
        return pulp.value(prob.objective)
    else:
        if DEBUG:
            print("LP did not find an optimal solution.")
        return -1


# n=9;k=4
# solve(n,k)

In [ ]:
import pandas as pd
import pickle
def ds(n,k, samples):
    df = pd.DataFrame()
    for i in range(samples):
        print(f'{i}/{samples}')
        P, results = solve(n,k)
        row_data = {f'val{i}': [results[i]] for i in range(len(results))}
        row_data['P'] = [P]
        df = pd.concat([df, pd.DataFrame(row_data)], ignore_index=True)

        with open(f'data_{n}_{k}_{samples}.pkl', 'wb') as f:
            pickle.dump(df, f)
    return df

In [ ]:
D = ds(9,4, 100)

0/100
solving n,k = (9, 4)
over m = [2, 3, 4, 5]
0 attempts reached
0 bounded solutions found
5000 attempts reached
107 bounded solutions found
10000 attempts reached
204 bounded solutions found
G:
 [[  1.   0.   0.   0.   7.  47. -92. -49.  56.]
 [  0.   1.   0.   0.  25. -30.  -8. -77. -52.]
 [  0.   0.   1.   0. -88.  25.  -7.  60.  38.]
 [  0.   0.   0.   1. -74. -27.  47.  58.  33.]]
Ψ [-1 -1 -1  1 -1]
a,b 7 1
X [4.0, 5.0, 6.0, 8.0]
Y [0, 2, 3]
1/100
solving n,k = (9, 4)
over m = [2, 3, 4, 5]
0 attempts reached
0 bounded solutions found
5000 attempts reached
105 bounded solutions found
G:
 [[  1.   0.   0.   0. -47.   6. -33. -85.  18.]
 [  0.   1.   0.   0.  95.  83. -70. -70.  68.]
 [  0.   0.   1.   0. -35.  60. -36. -95. -28.]
 [  0.   0.   0.   1.  18. -42.  36. -13. -50.]]
Ψ [-1 -1 -1 -1 -1]
a,b 7 3
X [4.0, 5.0, 6.0, 8.0]
Y [0, 1, 2]
2/100
solving n,k = (9, 4)
over m = [2, 3, 4, 5]
0 attempts reached
0 bounded solutions found
5000 attempts reached
112 bounded solutions found

In [10]:
print(D)

                                                    P        val1        val2  \
0   [[94, -95, 96, -52, 33], [85, -18, 13, 93, -70...   54.996822   53.154904   
1   [[-34, -30, -19, 16, 29], [70, -17, 22, 35, -4...    4.094484    4.805698   
2   [[95, 83, 63, 56, 73], [-89, 76, -18, -8, -23]...  130.366941  129.406725   
3   [[-9, 51, 72, 16, -18], [37, -9, -58, -12, 72]...   39.501072   39.338597   
4   [[-28, -89, 49, -18, -99], [47, 29, 63, -15, -...   97.032309   97.478420   
..                                                ...         ...         ...   
95  [[10, 49, -69, -65, 35], [-92, -31, -88, -43, ...  100.225813   98.008025   
96  [[-19, -56, 94, 85, -72], [80, 28, -30, 29, 54...   22.226162   44.262716   
97  [[-100, 50, -74, -18, -95], [95, 63, 29, 91, 3...  115.218388  119.056212   
98  [[2, -30, 89, 55, 37], [51, -2, 79, -50, 37], ...   19.529713   27.779706   
99  [[-32, -62, 63, 32, 3], [95, -12, -63, 62, 95]...    4.526065  124.477338   

          val3        val4 